In [1]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.19


In [4]:
# https://docs.sqlalchemy.org/en/20/tutorial/engine.html

from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [5]:
# Getting a connection
# https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.htmlhttps://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html

from sqlalchemy import text

with engine.connect() as conn:
    # textual SQL statement
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

    # The transaction is not committed automatically

2023-12-13 15:22:54,615 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 15:22:54,617 INFO sqlalchemy.engine.Engine select 'hello world'
2023-12-13 15:22:54,619 INFO sqlalchemy.engine.Engine [generated in 0.00417s] ()
[('hello world',)]
2023-12-13 15:22:54,622 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
# commit changes in a `as you go` fashion
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-12-13 15:25:23,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 15:25:23,113 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-12-13 15:25:23,114 INFO sqlalchemy.engine.Engine [generated in 0.00396s] ()
2023-12-13 15:25:23,116 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 15:25:23,117 INFO sqlalchemy.engine.Engine [generated in 0.00122s] [(1, 1), (2, 4)]
2023-12-13 15:25:23,118 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
# `begin once`` fashion
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-12-13 15:27:13,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 15:27:13,224 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-13 15:27:13,225 INFO sqlalchemy.engine.Engine [cached since 110.1s ago] [(6, 8), (9, 10)]
2023-12-13 15:27:13,226 INFO sqlalchemy.engine.Engine COMMIT


```
2023-12-13 15:27:13,219 INFO sqlalchemy.engine.Engine BEGIN (implicit)
```
BEGIN (implicit) means it did not actually send any command.

In [8]:
# select
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-12-13 15:29:48,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-13 15:29:48,156 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-13 15:29:48,157 INFO sqlalchemy.engine.Engine [generated in 0.00382s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-12-13 15:29:48,160 INFO sqlalchemy.engine.Engine ROLLBACK
